In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly
  pass
except Exception:
  pass

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd





# --- 1. Load the Dataset ---
# We load the tab-separated values (TSV) into pandas DataFrames
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

# --- 2. Data Preprocessing ---
# Convert labels to numbers: "ham" -> 0, "spam" -> 1
train_df["type"] = train_df["type"].map({"ham": 0, "spam": 1})
test_df["type"] = test_df["type"].map({"ham": 0, "spam": 1})

train_labels = train_df["type"].values
test_labels = test_df["type"].values

# --- 3. Tokenization ---
# This converts words into numbers the model can understand
vocab_size = 1000
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_df['msg'])

# Convert messages to sequences of numbers
train_sequences = tokenizer.texts_to_sequences(train_df['msg'])
test_sequences = tokenizer.texts_to_sequences(test_df['msg'])

# Pad the sequences so they are all the same length
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# --- 4. Build the Model ---
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# --- 5. Train the Model ---
history = model.fit(train_padded, train_labels, epochs=30, validation_data=(test_padded, test_labels), verbose=2)

In [ ]:
def predict_message(pred_text):
    # 1. Convert the text into a sequence of numbers (using the tokenizer from Cell 3)
    sequence = tokenizer.texts_to_sequences([pred_text])

    # 2. Pad the sequence so it fits the model's expected length
    sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # 3. Predict the probability (returns a value between 0 and 1)
    prediction = model.predict(sequence)[0][0]

    # 4. Assign the label: if probability >= 0.5 it's "spam", otherwise "ham"
    if prediction >= 0.5:
        label = "spam"
    else:
        label = "ham"

    # Return the list as expected by the test: [probability, label]
    return [prediction, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # 1. Convert the text into a sequence of numbers using the tokenizer
  sequence = tokenizer.texts_to_sequences([pred_text])

  # 2. Pad the sequence so it fits the model's expected length
  sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # 3. Predict the probability (returns a value between 0 and 1)
  prediction = model.predict(sequence)[0][0]

  # 4. Assign the label: if probability >= 0.5 it's "spam", otherwise "ham"
  if prediction >= 0.5:
      label = "spam"
  else:
      label = "ham"

  # Return the list as expected by the test: [probability, label]
  return [prediction, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
